# 7.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

- n_estimators - число деревьев в ансамбле
- max_depth - максимальная глубина каждого дерева в ансамбле
- subspaces_dim - размерность случайного подпространства для каждого дерева

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [ ]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]])
Y = np.array([1, 2, 3])
s = sample11(X, 2)

bootstrap_indices = s.bootstrap_sample(X)
X_sampled, y_sampled = s.get_subsample(X, Y, s.idx_subspace, bootstrap_indices)

3
2


In [ ]:
# Строки, выбранные из исходного массива X
bootstrap_indices

array([0, 1, 2])

In [ ]:
# Столбцы, выбранные из исходного массива X
s.idx_subspace

array([0, 2])

In [ ]:
# Матрица на выходе
X_sampled

array([[1., 3.],
       [4., 6.],
       [7., 9.]])

In [ ]:
# Вектор ответов на выходе
y_sampled

array([1., 2., 3.])

In [ ]:
class sample11(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)

    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X, random_state=42):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи бэггинга индексов.
        Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=424
        """
        return np.unique(np.random.choice(len(X), len(X), replace = True))

    @staticmethod
    def random_subspace(X, n_subspace, random_state=42):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи метода случайных подпространств признаков
        Количество этих признаков передается при помощи аргумента n_subspace
        Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
        """
        print(len(X[0]))
        print(n_subspace)
        return np.random.choice(len(X[0]), n_subspace, replace = False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled
        по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
        """
        X_sampled, y_sampled = np.array([]), np.array([])
        for id_obj in idx_obj:
            y_sampled = np.append(y_sampled, y[id_obj])
        for id_obj in idx_obj:
            for id_subspace in idx_subspace:
                X_sampled = np.append(X_sampled, X[id_obj][id_subspace])
        X_sampled = X_sampled.reshape((len(idx_obj), len(idx_subspace)))
        return X_sampled, y_sampled

In [ ]:

import numpy as np
np.random.seed(42)

from sklearn.tree import DecisionTreeClassifier

N_ESTIMATORS = 25
MAX_DEPTH = 15
SUBSPACE_DIM = 1


class sample2(object):
  def __init__(self, X, n_subspace):
    self.idx_subspace = self.random_subspace(X, n_subspace)

  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X, random_state=42):
    n = len(X)
    return np.unique(np.random.choice(n, n, replace=True))

  @staticmethod
  def random_subspace(X, n_subspace, random_state=42):
    return np.unique(np.random.choice(len(X[0]), n_subspace, replace=False))

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    X_sampled = []
    for i in idx_obj:
        X_sampled.append(np.array([X[i][j] for j in idx_subspace]))

    y_sampled = [y[i] for i in idx_obj]
    return np.array(X_sampled), y_sampled

class random_forest(object):
  def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.subspaces_dim = subspaces_dim
    self.random_state = random_state
    """
      Задайте все необходимые поля в рамках конструктора класса
    """

  def fit(self, X, y):
    self._estimators = []
    for i in range(self.n_estimators):
      """
        Напишите функцию обучения каждого из деревьев алгоритма на собственной подвыборке
      """
      self.s = sample(X, self.subspaces_dim)
      bootstrap_indices = self.s.bootstrap_sample(X)
      X_sampled, y_sampled = self.s.get_subsample(X, y, self.s.idx_subspace, bootstrap_indices)
      clf_tree = DecisionTreeClassifier(criterion="gini", max_depth=self.max_depth, random_state=self.random_state)
      clf_tree.fit(X_sampled, y_sampled)
      self._estimators.append(clf_tree)
      pass

  def predict(self, X):
    """
      Напишите функцию получения среднего предсказания алгоритма
    """
    predictions = np.array([])
    X_test = np.array([])
    for elem in X:
      for id in self.s.idx_subspace:
        X_test = np.append(X_test, elem[id])
    X_test = X_test.reshape((X.shape[0], len(self.s.idx_subspace)))

    predictions = []
    for estimator in self._estimators:
        yp = estimator.predict(X_test)
        predictions.append(yp.reshape(-1,1))

    ypred = np.mean(np.concatenate(predictions,axis=1),axis=1)
    return np.round(ypred).astype(int)

## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [ ]:
import numpy as np
X = np.array([[ 0.47819456, -1.57891216, -0.1018819 ,  1.11113501,  0.20826281,
        -1.11091227,  0.07844205],
       [ 0.11850997,  1.91073022,  0.95574903,  1.35798262,  0.56177995,
         0.26012021,  0.42404407],
       [-0.52304666,  0.75051167, -1.037804  , -0.10105312,  0.08559063,
         0.5102743 , -1.79068927],
       [-0.09078024,  1.62097709,  0.93284371,  1.0386902 , -0.68354252,
        -1.27138661,  0.15060651],
       [ 0.11676701, -0.71769062, -0.80119565,  0.73448495,  1.80728052,
         0.45770337,  0.20689119]])

In [ ]:
X.shape

(5, 7)

In [ ]:
X.shape[1]

7

In [ ]:
y = np.array([1, 2, 3, 4, 5])

In [ ]:
y.shape

(5,)

In [ ]:
X_test = np.array([[ 0.47819456, -1.57891216, -0.1018819 ,  1.11113501,  0.20826281,
        -1.11091227,  0.07844205],
       [ 0.11850997,  1.91073022,  0.95574903,  1.35798262,  0.56177995,
         0.26012021,  0.42404407],
       [-0.52304666,  0.75051167, -1.037804  , -0.10105312,  0.08559063,
         0.5102743 , -1.79068927],
       [-0.09078024,  1.62097709,  0.93284371,  1.0386902 , -0.68354252,
        -1.27138661,  0.15060651],
       [ 0.11676701, -0.71769062, -0.80119565,  0.73448495,  1.80728052,
         0.45770337,  0.20689119]])

In [ ]:
preds = rf.predict(X_test)

In [ ]:
preds

array([3., 5., 3., 3., 2.])

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics

iris = datasets.load_iris()

In [ ]:
X_train = iris["data"]
#X_train

In [ ]:
y_train = iris["target"]
#y_train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.3, shuffle=True, random_state=42
)

In [ ]:
N_ESTIMATORS = 50
MAX_DEPTH = 15
SUBSPACE_DIM = 3

In [ ]:
import numpy as np
np.random.seed(42)

from sklearn.tree import DecisionTreeClassifier

N_ESTIMATORS = 50
MAX_DEPTH = 15
SUBSPACE_DIM = 3


class sample(object):
  def __init__(self, X, n_subspace):
    self.idx_subspace = self.random_subspace(X, n_subspace)

  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X, random_state=42):
    n = len(X)
    return np.unique(np.random.choice(n, n, replace=True))

  @staticmethod
  def random_subspace(X, n_subspace, random_state=42):
    return np.unique(np.random.choice(len(X[0]), n_subspace, replace=False))

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    X_sampled = []
    for i in idx_obj:
        X_sampled.append(np.array([X[i][j] for j in idx_subspace]))

    y_sampled = [y[i] for i in idx_obj]
    return np.array(X_sampled), y_sampled

class random_forest(object):
  def __init__(self, n_estimators: int=100, max_depth: int=15, subspaces_dim: int=3, random_state: int=42):
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.subspaces_dim = subspaces_dim
    self.random_state = random_state
    """
      Задайте все необходимые поля в рамках конструктора класса
    """

  def fit(self, X, y):
    self._estimators = []
    self.idx_subspaces = []
    for i in range(self.n_estimators):
      """
        Напишите функцию обучения каждого из деревьев алгоритма на собственной подвыборке
      """
      self.s = sample(X, self.subspaces_dim)
      bootstrap_indices = self.s.bootstrap_sample(X)
      X_sampled, y_sampled = self.s.get_subsample(X, y, self.s.idx_subspace, bootstrap_indices)
      clf_tree = DecisionTreeClassifier(criterion="gini", max_depth=self.max_depth, random_state=self.random_state)
      clf_tree.fit(X_sampled, y_sampled)
      self._estimators.append(clf_tree)
      self.idx_subspaces.append(self.s.idx_subspace)
      pass

  def predict(self, X):
    """
      Напишите функцию получения среднего предсказания алгоритма
    """
    predictions = []
    for i, estimator in enumerate(self._estimators):
        idx = self.idx_subspaces[i]
        X_test = []
        for k in range(len(X)):
            X_test.append(np.array([X[k][j] for j in idx]))
        yp = estimator.predict(X_test)
        predictions.append(yp.reshape(-1,1))
    ypred = np.mean(np.concatenate(predictions,axis=1),axis=1)
    return np.round(ypred).astype(int)

In [ ]:
np.arange(1)

array([0])

In [ ]:
rf = random_forest(N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM, 42)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

1.0